# Data Analysis and Visualizations
---

In [20]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight')

# connect to database 
db_conn = sqlite3.connect('../../data/sql_db/spotify_data.db')

# create a function to run queries and output pandas dataframe
def Q(query, db=db_conn):
    return pd.read_sql(query, db)

## Check to see if all of our tables loaded.

In [21]:
Q('''
SELECT name
FROM sqlite_master;
  ''')

,name
0,Artist
1,Album
2,Track
3,Track_Feature


### As we can see from the output above all 4 tables loaded correctly.

## Create views
---

In [6]:
Q('''

''')

In [19]:
Q('''
PRAGMA table_info(Track_Feature);
''')

,cid,name,type,notnull,dflt_value,pk
0,0,track_id,TEXT,0,None,0
1,1,danceability,REAL,0,None,0
2,2,energy,REAL,0,None,0
3,3,instrumentalness,REAL,0,None,0
4,4,liveness,REAL,0,None,0
5,5,loudness,REAL,0,None,0
6,6,speechiness,REAL,0,None,0
7,7,tempo,REAL,0,None,0
8,8,type,TEXT,0,None,0
9,9,valence,REAL,0,None,0


In [ ]:
Q('''

''')